In [2]:
!unzip "/content/dataset.zip"

Archive:  /content/dataset.zip
replace SemEval2010_task8_all_data/SemEval2010_task8_scorer-v1.2/answer_key1.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: SemEval2010_task8_all_data/SemEval2010_task8_scorer-v1.2/answer_key1.txt  
  inflating: SemEval2010_task8_all_data/SemEval2010_task8_scorer-v1.2/answer_key2.txt  
  inflating: SemEval2010_task8_all_data/SemEval2010_task8_scorer-v1.2/answer_key3.txt  
  inflating: SemEval2010_task8_all_data/SemEval2010_task8_scorer-v1.2/answer_key5.txt  
  inflating: SemEval2010_task8_all_data/SemEval2010_task8_scorer-v1.2/proposed_answer1.txt  
  inflating: SemEval2010_task8_all_data/SemEval2010_task8_scorer-v1.2/proposed_answer2.txt  
  inflating: SemEval2010_task8_all_data/SemEval2010_task8_scorer-v1.2/proposed_answer3.txt  
  inflating: SemEval2010_task8_all_data/SemEval2010_task8_scorer-v1.2/proposed_answer4.txt  
  inflating: SemEval2010_task8_all_data/SemEval2010_task8_scorer-v1.2/proposed_answer5.txt  
  inflating: SemEval2010_task8

In [3]:
import re
import csv

In [4]:
training_data_filepath = "/content/SemEval2010_task8_all_data/SemEval2010_task8_training/TRAIN_FILE.TXT"
testing_data_filepath = "/content/SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT"

In [5]:
def read_data_from_file(filepath):
  data = []
  with open(filepath, 'r') as file:
    lines = file.readlines()
    for line in lines:
      line = line.strip()
      if line:
        data.append(line)
  return data

training_data = read_data_from_file(training_data_filepath)
testing_data = read_data_from_file(testing_data_filepath)


In [6]:
def preprocess_data(data):
  processed_data = []
  for line in data:
    processed_data.append(line.split('|'))
  for line in processed_data:
    if re.findall(r"^Comment", line[0]):
      processed_data.remove(line)
  sentences = []
  relations = []
  for i in range(len(processed_data)):
    if i%2 == 0:
      sentences.append(processed_data[i])
    else:
      relations.append(processed_data[i])
  #sentences
  processed_sentences = []
  for sentence in sentences:
    #removing the number in front of the sentence    
    sentence[0] = re.sub(r"\d+\t", '',sentence[0])
    #removing double quotes
    sentence[0] = re.sub('"', "", sentence[0])
    #removing entity indicators
    sentence[0] = re.sub(r"<e1>|<e2>|</e1>|</e2>", "", sentence[0])
    processed_sentences.append(sentence[0])
  #encoding relations
  encoded_relations = []
  encoding = {"Cause-Effect": 0, "Component-Whole": 1, "Product-Producer": 2, "Other": 3}
  for relation in relations:
    if "Cause-Effect" in relation[0]:
      encoded_relations.append(0)
    elif "Component-Whole" in relation[0]:
      encoded_relations.append(1)
    elif "Product-Producer" in relation[0]:
      encoded_relations.append(2)
    else:
      encoded_relations.append(3)
  return processed_sentences, encoded_relations

corpus, labels = preprocess_data(training_data)
testing_corpus, testing_labels = preprocess_data(testing_data)

In [7]:
print("Data Statistics")
frequency_of_labels = {"Cause-Effect": 0, "Component-Whole": 0, "Product-Producer": 0, "Other": 0}
for label in labels:
  if label == 0:
    frequency_of_labels["Cause-Effect"]+=1
  elif label == 1:
    frequency_of_labels["Component-Whole"]+=1
  elif label == 2:
    frequency_of_labels["Product-Producer"]+=1
  else:
    frequency_of_labels["Other"]+=1

for label in frequency_of_labels.keys():
  print(label, ": ", (frequency_of_labels[label]/len(labels))*100, "%\n", end="")

Data Statistics
Cause-Effect :  12.537499999999998 %Component-Whole :  11.7625 %Product-Producer :  8.9625 %Other :  66.73750000000001 %

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(corpus)
X = X.toarray()

In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np
parameters =  {'kernel':['linear', 'rbf'], 'C':[1, 10], 'decision_function_shape': ['ovo']}
model = GridSearchCV(SVC(), parameters, verbose = 3, cv = 4)
model.fit(X, labels)

Fitting 4 folds for each of 4 candidates, totalling 16 fits
[CV 1/4] END C=1, decision_function_shape=ovo, kernel=linear;, score=0.765 total time= 1.9min
[CV 2/4] END C=1, decision_function_shape=ovo, kernel=linear;, score=0.775 total time= 1.8min
[CV 3/4] END C=1, decision_function_shape=ovo, kernel=linear;, score=0.759 total time= 1.9min
[CV 4/4] END C=1, decision_function_shape=ovo, kernel=linear;, score=0.684 total time= 1.7min
[CV 1/4] END C=1, decision_function_shape=ovo, kernel=rbf;, score=0.745 total time= 3.0min
[CV 2/4] END C=1, decision_function_shape=ovo, kernel=rbf;, score=0.741 total time= 3.0min
[CV 3/4] END C=1, decision_function_shape=ovo, kernel=rbf;, score=0.744 total time= 3.0min
[CV 4/4] END C=1, decision_function_shape=ovo, kernel=rbf;, score=0.693 total time= 2.9min
[CV 1/4] END C=10, decision_function_shape=ovo, kernel=linear;, score=0.750 total time= 1.8min
[CV 2/4] END C=10, decision_function_shape=ovo, kernel=linear;, score=0.758 total time= 1.8min
[CV 3/4] E

GridSearchCV(cv=4, estimator=SVC(),
             param_grid={'C': [1, 10], 'decision_function_shape': ['ovo'],
                         'kernel': ['linear', 'rbf']},
             verbose=3)

In [11]:
from sklearn.metrics import classification_report
X_test = vectorizer.fit_transform(testing_corpus)
X_test = X_test.toarray()
prediction = model.predict(X_test)
target_names = ["Cause-Effect", "Component-Whole", "Product-Producer", "Other"]
print(classification_report(testing_labels, prediction, target_names = target_names))

                  precision    recall  f1-score   support

    Cause-Effect       0.00      0.00      0.00       328
 Component-Whole       0.33      0.01      0.02       312
Product-Producer       0.14      0.00      0.01       231
           Other       0.68      0.99      0.81      1846

        accuracy                           0.68      2717
       macro avg       0.29      0.25      0.21      2717
    weighted avg       0.51      0.68      0.55      2717

